## 注意(attention)！开始做前必读项！

- 所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件
- 对于提供的数据集，不要改存储地方，也不要修改文件名和内容
- 确保到时候git pull之后我们可以直接运行这个 starter_code文件
- 不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块
- 写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！
- 第一次作业很重要，一定要完成！ 相信会有很多的收获！

## Part 1: 搭建一个分词工具

### Part 1.1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [53]:
# 第一步： 从dic.txt中读取所有中文词。
# hint: 思考一下用什么数据结构来存储这个词典会比较好？ 要考虑我们每次查询一个单词的效率。 
import xlrd
workbook = xlrd.open_workbook("./data/综合类中文词库.xlsx")
worksheet = workbook.sheet_by_index(0)
col_data = worksheet.col_values(0)
col_index = [0.00001] * len(col_data)
dic_words = dict(zip(col_data,col_index))   # 保存词典库中读取的单词

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}

In [209]:
#  分数（10）
## 请编写word_segment_naive函数来实现对输入字符串的分词
def word_segment_naive(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    segments = [] 
    # 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。 
    # 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
    # 格式为：segments = [["今天"，“天气”，“好”],["今天"，“天“，”气”，“好”],["今“，”天"，“天气”，“好”],...]

    if len(input_str) == 0:
        return segments

    for idx in range(1,len(input_str)+1):
        word = input_str[0:idx]
        if word in dic_words or word in word_prob:
            sub_string = word_segment_naive(input_str[idx:])
           
            if (len(sub_string) == 0) & (len(input_str[idx:]) == 0):
                segments.append([word])
            else:
                for seg in sub_string:
                    seg = [word] + seg
                    segments.append(seg)
    return segments


In [67]:
# 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
import numpy as np
import sys
def best_seg(segments):
    best_segment = []
    best_score = sys.maxsize
    for seg in segments:
        score = 0
        for item in seg:
            if item in word_prob:
                score -= np.log(word_prob[item])
            else:
                score -= np.log(dic_words[item])
        if score < best_score:
            best_score = score
            best_segment = seg
    return best_segment      

In [211]:
# 测试
print (best_seg(word_segment_naive("北京的天气真好啊")))
print (best_seg(word_segment_naive("今天的课程内容很有意思")))
print (best_seg(word_segment_naive("经常有意见分歧")))

['北京', '的', '天气', '真好啊']
['今天', '的', '课程', '内容', '很有', '意思']
['经常', '有意见', '分歧']


### Part 1.2  基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph） 参考：课程内容
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
注意：思考用什么方式来存储这种有向图比较合适？ 不一定只有一种方式来存储这种结构。 

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
具体算法参考课程中讲过的内容

#### Step 3: 返回结果
跟PART 1.1的要求一致

In [256]:
# 分数（10）


import sys
import numpy as np

sys.setrecursionlimit(9000000)
    ## 请编写word_segment_viterbi函数来实现对输入字符串的分词
    
def word_segment_viterbi(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    """ 
    if len(input_str) == 0:
        return 0
    
    # 第一步： 构建图                 
    n = len(input_str)
    path = np.zeros([n+1,n+1])
    
    for i in range(n):
        if input_str[i] in word_prob:
            path[i,i+1] = word_prob[input_str[i]]
        elif input_str[i] in dic_words:
            path[i,i+1] = 0.00001
        for j in range(i+1,n+1):
            if input_str[i:j] in word_prob:
                path[i,j] = word_prob[input_str[i:j]]
            elif input_str[i:j] in dic_words:
                    path[i,i+1] = 0.00001

    
    # 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    # hint: 思考为什么不用相乘: p(w1)p(w2)...而是使用negative log sum:  -log(w1)-log(w2)-...  
    minPath = np.zeros(n+1)
    score = [sys.maxsize]*(n+1)
    score[0] = 0
    seg = []
    for i in range(1,n+1):
        p = 0
        for j in range(0,n):
            if path[j,i] != 0:
                if score[i] > -np.log(path[j,i])+score[j]:
                    score[i] = -np.log(path[j,i])+score[j]
                    minPath[i] = j
                     
    # 第三步： 根据最好的PATH, 返回最好的切分

    i = n
    while i > 0:
        seg.append(input_str[int(minPath[i]):i])
        i = int(minPath[i])
    seg = seg[::-1]
    return seg

In [257]:
# 测试
print (word_segment_viterbi("北京的天气真好啊"))
print (word_segment_viterbi("今天的课程内容很有意思"))
print (word_segment_viterbi("经常有意见分歧"))

['北京', '的', '天气', '真好啊']
['今天', '的', '课程', '内容', '很有', '意思']
['经常', '有意见', '分歧']


In [258]:
# 分数（3）
# TODO: 第一种方法和第二种方法的时间复杂度和空间复杂度分别是多少？
# 第一个方法： 
# 时间复杂度= , 空间复杂度=

# 第二个方法：
# 时间复杂度= , 空间复杂度=

## Part 2:  搭建一个简单的问答系统

本次项目的目标是搭建一个基于检索式的简单的问答系统。至于什么是检索式的问答系统请参考课程直播内容/PPT介绍。 

通过此项目，你将会有机会掌握以下几个知识点：
1. 字符串操作   2. 文本预处理技术（词过滤，标准化）   3. 文本的表示（tf-idf, word2vec)  4. 文本相似度计算  5. 文本高效检索

此项目需要的数据：
1. dev-v2.0.json: 这个数据包含了问题和答案的pair， 但是以JSON格式存在，需要编写parser来提取出里面的问题和答案。 
2. glove.6B: 这个文件需要从网上下载，下载地址为：https://nlp.stanford.edu/projects/glove/， 请使用d=100的词向量

##### 检索式的问答系统
问答系统所需要的数据已经提供，对于每一个问题都可以找得到相应的答案，所以可以理解为每一个样本数据是 <问题、答案>。 那系统的核心是当用户输入一个问题的时候，首先要找到跟这个问题最相近的已经存储在库里的问题，然后直接返回相应的答案即可。 举一个简单的例子：

假设我们的库里面已有存在以下几个<问题,答案>：
<"贪心学院主要做什么方面的业务？”， “他们主要做人工智能方面的教育”>
<“国内有哪些做人工智能教育的公司？”， “贪心学院”>
<"人工智能和机器学习的关系什么？", "其实机器学习是人工智能的一个范畴，很多人工智能的应用要基于机器学习的技术">
<"人工智能最核心的语言是什么？"， ”Python“>
.....

假设一个用户往系统中输入了问题 “贪心学院是做什么的？”， 那这时候系统先去匹配最相近的“已经存在库里的”问题。 那在这里很显然是 “贪心学院是做什么的”和“贪心学院主要做什么方面的业务？”是最相近的。 所以当我们定位到这个问题之后，直接返回它的答案 “他们主要做人工智能方面的教育”就可以了。 所以这里的核心问题可以归结为计算两个问句（query）之间的相似度。

在本次项目中，你会频繁地使用到sklearn这个机器学习库。具体安装请见：http://scikit-learn.org/stable/install.html  sklearn包含了各类机器学习算法和数据处理工具，包括本项目需要使用的词袋模型，均可以在sklearn工具包中找得到。 另外，本项目还需要用到分词工具jieba, 具体使用方法请见 https://github.com/fxsjy/jieba

### Part 2.1  第一部分： 读取文件，并把内容分别写到两个list里（一个list对应问题集，另一个list对应答案集）

In [91]:
import json
def read_corpus():
    """
    读取给定的语料库，并把问题列表和答案列表分别写入到 qlist, alist 里面。 在此过程中，不用对字符换做任何的处理（这部分需要在 Part 2.3里处理）
    qlist = ["问题1"， “问题2”， “问题3” ....]
    alist = ["答案1", "答案2", "答案3" ....]
    务必要让每一个问题和答案对应起来（下标位置一致）
    """
    qlist = []
    alist = []
    f = open('./data/train-v2.0.json')
    r = f.read()
    data = json.loads(r)
    for item in data["data"]:
        for item1 in item['paragraphs']:
            for item2 in item1['qas']:
                qlist.append(item2['question'].lower())
                if item2['answers'] == []:
                    alist.append('')
                else:
                    alist.append(item2['answers'][0]['text'].lower())
    
    assert len(qlist) == len(alist)  # 确保长度一样
    return qlist, alist

### Part 2.2 理解数据（可视化分析/统计信息）
对数据的理解是任何AI工作的第一步，需要充分对手上的数据有个更直观的理解。 

In [93]:
# 分数（10）
# TODO: 统计一下在qlist 总共出现了多少个单词？ 总共出现了多少个不同的单词？
#       这里需要做简单的分词，对于英文我们根据空格来分词即可，其他过滤暂不考虑（只需分词）
import nltk
qlist, alist = read_corpus()
aword ,qword= [] ,[]
for item in qlist:
    for item1 in nltk.word_tokenize(item):
        qword.append(item1)
for item in alist:
    for item1 in nltk.word_tokenize(item):
        aword.append(item1)
word_total = len(aword)
afdist = nltk.FreqDist(aword)
qfdist = nltk.FreqDist(qword)
print (word_total)
print (len(afdist))

293024
42693


In [76]:
# TODO: 在qlist和alist里出现次数最多的TOP 10单词分别是什么？ 
print(afdist.most_common(10))
print(qfdist.most_common(10))


[('the', 13275), ('of', 8902), ('and', 8187), (',', 7543), ('to', 3459), ('a', 3323), ('in', 2873), ('.', 1856), ('The', 1796), ("'s", 1414)]
[('?', 129114), ('the', 88430), ('What', 59300), ('of', 49022), ('in', 32399), ('to', 27166), ('was', 24936), ('is', 24466), ('did', 23310), ('what', 18673)]


### 2.3 文本预处理
次部分需要尝试做文本的处理。在这里我们面对的是英文文本，所以任何对英文适合的技术都可以考虑进来。

In [96]:
# 分数（10）

# TODO: 对于qlist, alist做文本预处理操作。 可以考虑以下几种操作：
#       1. 停用词过滤 （去网上搜一下 "english stop words list"，会出现很多包含停用词库的网页，或者直接使用NLTK自带的）   
#       2. 转换成lower_case： 这是一个基本的操作   
#       3. 去掉一些无用的符号： 比如连续的感叹号！！！， 或者一些奇怪的单词。
#       4. 去掉出现频率很低的词：比如出现次数少于10,20....
#       5. 对于数字的处理： 分词完只有有些单词可能就是数字比如44，415，把所有这些数字都看成是一个单词，这个新的单词我们可以定义为 "#number"
#       6. stemming（利用porter stemming): 因为是英文，所以stemming也是可以做的工作
#       7. 其他（如果有的话）
#       请注意，不一定要按照上面的顺序来处理，具体处理的顺序思考一下，然后选择一个合理的顺序
#  hint: 停用词用什么数据结构来存储？ 不一样的数据结构会带来完全不一样的效率！ 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def do(inlist,infdist):
    q = []
    for sent in inlist:
        s = []
        w = nltk.word_tokenize(sent)
        for word in w:
            if infdist[word] < 10 and word not in stop_words:
                    if (word.isdigit() or word.isalnum()) and not word.isalpha:
                        s.append("#number")
                    else:
                        s.append(porter_stemmer.stem(word))
        q.append(s)
    return q
# a = [porter_stemmer.stem(w.lower()) for w in aword if not w in stop_words]
# q = [porter_stemmer.stem(w.lower()) for w in qword if not w in stop_words]
q, a = do(qlist,qfdist), do(alist,afdist)   # 更新后的

[[], ['sing', 'danc'], [], [], [], [], ['danger'], ['mathew', 'knowl'], [], ['singer'], ['danger'], [], [], ['singer'], ['danger'], [], [], [], ['dreamgirl'], [], [], ['cadillac'], [], ["b'day"], ['dreamgirl'], [], ['sasha', 'fierc'], ['monogami'], [], [], [], [], ['modern-day', 'feminist'], [], [], [], [], [], [], [], ['maiden'], [], [], ['xerox'], ['hairdress', 'salon'], ['solang'], ['broussard'], ['xerox'], ['salon'], ['solang'], ['broussard'], [], ['fredericksburg'], ['darlett'], [], ['instructor', 'darlett'], [], ['magnet'], ['imagin'], ['fredericksburg'], ['darlett'], [], [], ['arn', 'frager'], [], ['elektra'], ['arn', 'frager'], [], [], ['elektra'], [], [], ['girl', 'tyme'], ['arn', 'frager'], [], ['dwayn', 'wiggin', 'grass'], [], ['say'], ['marc'], [], ['isaiah'], [], ['say'], ['marc'], ['isaiah'], [], [], [], ['marc'], [], ['boyfriend'], [], ['luckett', 'rober'], ['coupl'], [], ['farrah', 'michel'], [], [], ['farrah'], [], [], [], ['663,000'], ['georg', 'bizet'], ['survivor'],

In [78]:
# TODO: 在前面步骤里，我们删除了出现次数比较少的单词，那你选择的阈值是多少（小于多少的去掉？）， 这个阈值是根据什么来选择的？ 

### 2.4 文本表示
当我们做完关键的预处理过程之后，就需要把每一个文本转换成向量。

In [ ]:
# 分数（10）

# TODO: 把qlist中的每一个问题字符串转换成tf-idf向量, 转换之后的结果存储在X矩阵里。 X的大小是： N* D的矩阵。 这里N是问题的个数（样本个数），
#       D是字典库的大小。 

vectorizer =  # 定义一个tf-idf的vectorizer

X =   # 结果存放在X矩阵

In [ ]:
# TODO: 矩阵X有什么特点？ 计算一下它的稀疏度

print (sparsity)  # 打印出稀疏度(sparsity)

### 2.5 对于用户的输入问题，找到相似度最高的TOP5问题，并把5个潜在的答案做返回

In [ ]:
# 分数（10）

def top5results(input_q):
    """
    给定用户输入的问题 input_q, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 对于用户的输入 input_q 首先做一系列的预处理，然后再转换成tf-idf向量（利用上面的vectorizer)
    2. 计算跟每个库里的问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    
    top_idxs = []  # top_idxs存放相似度最高的（存在qlist里的）问题的下表 
                   # hint: 利用priority queue来找出top results. 思考为什么可以这么做？ 
    
    return alist[top_idxs]  # 返回相似度最高的问题对应的答案，作为TOP5答案    

In [ ]:
# TODO: 编写几个测试用例，并输出结果
print (top5results(""))
print (top5results(""))

In [ ]:
# 分数（5）

# TODO: 上面的top5results算法的时间复杂度和空间复杂度分别是多少？

时间复杂度 = O()， 空间复杂度 = O()

### 2.6 利用倒排表的优化。 
上面的算法，一个最大的缺点是每一个用户问题都需要跟库里的所有的问题都计算相似度。假设我们库里的问题非常多，这将是效率非常低的方法。 这里面一个方案是通过倒排表的方式，先从库里面找到跟当前的输入类似的问题描述。然后针对于这些candidates问题再做余弦相似度的计算。这样会节省大量的时间。

In [ ]:
# 分数（10）

# TODO: 基于倒排表的优化。在这里，我们可以定义一个类似于hash_map, 比如 inverted_index = {}， 然后存放包含每一个关键词的文档出现在了什么位置，
#       也就是，通过关键词的搜索首先来判断包含这些关键词的文档（比如出现至少一个），然后对于candidates问题做相似度比较。
# 
inverted_idx = {}  # 定一个一个简单的倒排表
def top5results_invidx(input_q):
    """
    给定用户输入的问题 input_q, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate
    2. 对于用户的输入 input_q 首先做一系列的预处理，然后再转换成tf-idf向量（利用上面的vectorizer)
    3. 计算跟每个库里的问题之间的相似度
    4. 找出相似度最高的top5问题的答案
    """

In [ ]:
# TODO: 编写几个测试用例，并输出结果
print (top5results_invidx(""))
print (top5results_invidx(""))

In [ ]:
# 分数（3）

# TODO: 上面的top5results算法的时间复杂度和空间复杂度分别是多少？

时间复杂度 = O()， 空间复杂度 = O()

### 2.7 基于词向量的文本表示
上面所用到的方法论是基于词袋模型（bag-of-words model）。这样的方法论有两个主要的问题：1. 无法计算词语之间的相似度  2. 稀疏度很高。 在2.7里面我们
讲采用词向量作为文本的表示。词向量方面需要下载： https://nlp.stanford.edu/projects/glove/ （请下载glove.6B.zip），并使用d=100的词向量（100维）。


In [ ]:
# 分数（10）

# TODO
emb = # 读取每一个单词的嵌入。这个是 D*H的矩阵，这里的D是词典库的大小， H是词向量的大小。 这里面我们给定的每个单词的词向量，那句子向量怎么表达？
      # 其中，最简单的方式 句子向量 = 词向量的平均（出现在问句里的）， 如果给定的词没有出现在词典库里，则忽略掉这个词。

def top5results_emb(input_q):
    """
    给定用户输入的问题 input_q, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate
    2. 对于用户的输入 input_q，转换成句子向量
    3. 计算跟每个库里的问题之间的相似度
    4. 找出相似度最高的top5问题的答案
    """
    


In [ ]:
# TODO: 编写几个测试用例，并输出结果
print (top5results_emb(""))
print (top5results_emb(""))

# 我们在验收作业时在后台会建立几个测试用例，来验证返回的准确性。

### 2.8 做完本次项目做完之后有什么收获？ 

#分数（2）

回答 = “”